In [1]:
import pandas as pd
import glob as gb
import os

files = gb.glob('./data/Ward_Age/*')
dataname = '町丁目名年齢年代別人口分布'

d=os.getcwd()
prefix = d + '/../data/Ward_Age/'

fiveband = ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
 '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
 '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳',
 '105~109歳', '110~114歳', '115~119歳', '120~124歳', '125~129歳']
# c
#g_code = pd.read_csv(prefix+'東京都人口世帯数_estat.txt', encoding='SHIFT-JIS', dtype={'KEY_CODE':str})
#g_code[g_code['HYOSYO']>3]
#g_code=g_code[['KEY_CODE','NAME','CITYNAME']]
wds = ['市区町村コード','市区町村名','大字町丁目コード','大字町丁目名']
c_code = pd.read_csv(prefix + '13_2020.csv', encoding='utf-8', dtype={'大字町丁目コード':str})[wds]

In [2]:
d2k = {
    '１丁':'一丁',
    '２丁':'二丁',
    '３丁':'三丁',
    '４丁':'四丁',
    '５丁':'五丁',
    '６丁':'六丁',
    '７丁':'七丁',
    '８丁':'八丁',
    '９丁':'九丁'
}

sd2k = {
    '1丁':'１丁',
    '2丁':'２丁',
    '3丁':'３丁',
    '4丁':'４丁',
    '5丁':'５丁',
    '6丁':'６丁',
    '7丁':'７丁',
    '8丁':'８丁',
    '9丁':'９丁'
}

k2d = {v: k for k, v in d2k.items()}

## 6. 千代田区

In [3]:
achy = pd.read_excel(prefix + '千代田区2023年1月.xlsx')

achy = achy.set_axis(['a','b','c','d','e','f','g','h','i','j','k'],axis=1)

achy = achy[~achy['a'].isnull()]
area = list(achy[achy['b'].isnull()]['a'])

data = achy[~achy['b'].isnull()][['a','b','e','g','h','k']].reset_index(drop=True)

kk = dict()

for i in range(0,115):
    left = data.loc[(i*58+1):(i*58+57),['a','b','e']].astype({'a':int,'e':int})
    right = data.loc[(i*58):(i*58+54), ['g','h','k']].astype({'g':int,'k':int})
    left = left.set_axis(['年齢','生年','人口'],axis=1)
    right = right.set_axis(['年齢','生年','人口'],axis=1)
    kk[area[i]] = pd.concat([left,right])

q = pd.DataFrame()

for v in area:
    kk[v]['町丁目名']=v
    q = pd.concat([q,kk[v]],ignore_index=True)

q['区']='千代田区'
q = q.drop(columns='生年')

for i in k2d.keys():
    q['町丁目名']=q['町丁目名'].str.replace(i,k2d[i])

ch_fd = pd.merge(q, 
                 c_code, 
                 left_on=['区','町丁目名'], 
                 right_on=['市区町村名','大字町丁目名'],
#                 how='left'
                )
ch_fd.head()

,年齢,人口,町丁目名,区,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名
0,0,0,丸の内１丁目,千代田区,13101,千代田区,131010011001,丸の内１丁目
1,1,0,丸の内１丁目,千代田区,13101,千代田区,131010011001,丸の内１丁目
2,2,0,丸の内１丁目,千代田区,13101,千代田区,131010011001,丸の内１丁目
3,3,0,丸の内１丁目,千代田区,13101,千代田区,131010011001,丸の内１丁目
4,4,0,丸の内１丁目,千代田区,13101,千代田区,131010011001,丸の内１丁目


## 7.練馬区 

In [4]:
nemmy = pd.read_excel(prefix + '練馬区2023年1月.xlsx')

#nemmy[nemmy['男']=='女']
nemmy = nemmy.rename(columns={'男':'年齢'})

nem_m = nemmy.loc[0:110,:]
nem_f = nemmy.loc[142:252,:]

nem_m = nem_m.set_index('年齢')
nem_f = nem_f.set_index('年齢')

In [5]:
colsm = ['年齢','町丁目名','人口']
n_f = pd.DataFrame(nem_f.stack()).reset_index().set_axis(colsm,axis=1)
n_m = pd.DataFrame(nem_m.stack()).reset_index().set_axis(colsm,axis=1)

n_f = n_f.astype({'人口':int})
n_m = n_m.astype({'人口':int})
n = pd.merge(n_m,n_f,on=['年齢','町丁目名'])

n['人口']=n['人口_x']+n['人口_y']
n['区']='練馬区'
ner=n[['区','町丁目名','年齢','人口']]

In [6]:
ner_fd = pd.merge(ner,
                  c_code,
                  left_on=['区','町丁目名'],
                  right_on=['市区町村名','大字町丁目名'],
#                  how='left'
                 )
ner_fd.head()

,区,町丁目名,年齢,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名
0,練馬区,旭丘１丁目,0,25,13120,練馬区,131200021001,旭丘１丁目
1,練馬区,旭丘１丁目,1,22,13120,練馬区,131200021001,旭丘１丁目
2,練馬区,旭丘１丁目,2,37,13120,練馬区,131200021001,旭丘１丁目
3,練馬区,旭丘１丁目,3,19,13120,練馬区,131200021001,旭丘１丁目
4,練馬区,旭丘１丁目,4,21,13120,練馬区,131200021001,旭丘１丁目


## 8. 世田谷区

In [7]:
def qqq(st):
    cols = [0]+list(range(1,100,3))
    a=st.iloc[:,1:100:3]
    b=st.iloc[:,0]
    c=pd.concat([b,a],axis=1)
    cc=c.loc[1:,:]
    #c.loc[1:,:]
    q=cc[cc['R5.1.1'].str.contains('～', na=False)].set_index('R5.1.1')
    qa=cc[cc['R5.1.1'].str.contains('以上', na=False)].set_index('R5.1.1')
    q=pd.concat([q,qa])
    qq=pd.DataFrame(q.stack().reset_index())
    return qq.rename(columns={'R5.1.1':'世代',
                              'level_1':'町丁目名',
                              0:'人口'})

In [8]:
st_fd = pd.DataFrame()
for i in list(range(1,6)):
    st = pd.read_excel(prefix +'世田谷2023年1月'+str(i)+'.xls')
    st_fd = pd.concat([st_fd, qqq(st)])

st_fd['町丁目名'] = st_fd['町丁目名'].str.replace('　','', regex=True)

# 池尻４丁目は1番から32番と、33番から39番が泣き別れている。Excelで統合作業を行う。

st_fd['区']='世田谷区'
st_fd = pd.merge(st_fd,
                 c_code,
                 left_on=['区','町丁目名'],
                 right_on=['市区町村名','大字町丁目名'],
#                 how='left'
                )

jedi = list(st_fd['世代'])[0:21]
fb = fiveband[0:21]
j2f = dict(zip(jedi,fb))

for i,r in st_fd.iterrows():
    st_fd.loc[i,'世代'] = j2f[r.世代]

In [9]:
st_fd

,世代,町丁目名,人口,区,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名
0,0~4歳,池尻１丁目,73,世田谷区,13112,世田谷区,131120036001,池尻１丁目
1,5~9歳,池尻１丁目,82,世田谷区,13112,世田谷区,131120036001,池尻１丁目
2,10~14歳,池尻１丁目,95,世田谷区,13112,世田谷区,131120036001,池尻１丁目
3,15~19歳,池尻１丁目,83,世田谷区,13112,世田谷区,131120036001,池尻１丁目
4,20~24歳,池尻１丁目,76,世田谷区,13112,世田谷区,131120036001,池尻１丁目
...,...,...,...,...,...,...,...,...
3460,80~84歳,北烏山３丁目,157,世田谷区,13112,世田谷区,131120016003,北烏山３丁目
3461,85~89歳,北烏山３丁目,136,世田谷区,13112,世田谷区,131120016003,北烏山３丁目
3462,90~94歳,北烏山３丁目,56,世田谷区,13112,世田谷区,131120016003,北烏山３丁目
3463,95~99歳,北烏山３丁目,16,世田谷区,13112,世田谷区,131120016003,北烏山３丁目


## 1. 大田区

In [10]:
ohta_file = prefix + '大田区2022年1月_rev.xlsx'

ohta_data = pd.read_excel(ohta_file).set_index('年齢')

ohta_finedata = pd.DataFrame(ohta_data.stack()).reset_index().set_axis(['年齢','町丁目名','人口'],axis=1)

ohta_finedata = ohta_finedata[['町丁目名','年齢','人口']]

ohta_finedata['町丁目名'] = ohta_finedata['町丁目名'].str.replace('　','')

ohta_finedata = pd.merge(ohta_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

ohta_finedata['区']='大田区'

for i in d2k.keys():
    ohta_finedata['町丁目名']=ohta_finedata['町丁目名'].str.replace(i,d2k[i])


In [11]:
ohta_finedata.head()

,町丁目名,年齢,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区
0,大森東一丁目,0,24,13111,大田区,131110039001,大森東１丁目,大田区
1,大森東一丁目,1,28,13111,大田区,131110039001,大森東１丁目,大田区
2,大森東一丁目,2,36,13111,大田区,131110039001,大森東１丁目,大田区
3,大森東一丁目,3,36,13111,大田区,131110039001,大森東１丁目,大田区
4,大森東一丁目,4,31,13111,大田区,131110039001,大森東１丁目,大田区


In [12]:
#ohta_finedata.to_excel(prefix + dataname + '（大田）.xlsx', index=False)

## 2. 品川

In [13]:
male=list(range(0,241,2))
female=list(range(1,241,2))

shina_file = prefix + '品川区2023年1月_rev.xls'
shina_data = pd.read_excel(shina_file, dtype={'行政コード':int})

shina_m = shina_data.loc[:,['行政コード','丁目名']+male].set_index(['行政コード','丁目名']).stack()
shina_f = shina_data.loc[:,['行政コード','丁目名']+female].set_index(['行政コード','丁目名']).stack()

shina_m_fine= pd.DataFrame(shina_m).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_m_fine['年齢']=shina_m_fine['age']//2

shina_f_fine= pd.DataFrame(shina_f).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_f_fine['年齢']=(shina_f_fine['age'])//2

scols=['丁目名','年齢','人口']
mer=['丁目名','年齢']

shina_m_fine=shina_m_fine.drop(columns=['age'])[scols]
shina_f_fine=shina_f_fine.drop(columns=['age'])[scols]


shina_finedata = pd.merge(shina_m_fine, shina_f_fine, on=mer)
shina_finedata['区']='品川区'

#for t,i in shina_finedata.iterrows():
#    shina_finedata.loc[i,'丁目名'] = shina_finedata.loc[i, '丁目名'].replace('\u3000','')
#    print(shina_finedata.loc[i, '丁目名'])

shina_finedata['人口'] = shina_finedata['人口_x']+shina_finedata['人口_y']
shina_finedata = shina_finedata.drop(columns=['人口_x','人口_y']).reset_index(drop=True)
shina_finedata = shina_finedata.rename(columns={'丁目名':'町丁目名'})
shina_finedata = pd.merge(shina_finedata, c_code, left_on=['区','町丁目名'], right_on=['市区町村名','大字町丁目名'])



for i in d2k.keys():
    shina_finedata['町丁目名']=shina_finedata['町丁目名'].str.replace(i,d2k[i])


In [14]:
shina_finedata.head()

,町丁目名,年齢,区,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名
0,北品川一丁目,0,品川区,20,13109,品川区,131090019001,北品川１丁目
1,北品川一丁目,1,品川区,17,13109,品川区,131090019001,北品川１丁目
2,北品川一丁目,2,品川区,21,13109,品川区,131090019001,北品川１丁目
3,北品川一丁目,3,品川区,18,13109,品川区,131090019001,北品川１丁目
4,北品川一丁目,4,品川区,14,13109,品川区,131090019001,北品川１丁目


## 3. 目黒

In [15]:
meg_file = prefix +'目黒区2022年12月_rev.xlsx'
meg_data = pd.read_excel(meg_file)
#meg_data

top = meg_data[meg_data['a']=='町丁別年齢別人口表'].reset_index()['index']
end = meg_data[meg_data['a']=='104以上'].reset_index()['index']
#print(list(top))
#print(list(end))

meg_tmp = pd.DataFrame()

#meg_data['町丁目名']=''

for k in top:
    chomei = meg_data.iloc[k+1,0].replace('　','')

    for j in list(range(k+4,k+129)):
        meg_data.loc[j,'町丁目名'] = chomei

    chodf = meg_data.iloc[(k+4):(k+129),:]
    
    chodf = chodf[~(chodf['a'].str.contains('～',na=False))]
    chodf = chodf[~(chodf['a'].str.contains('以上',na=False))]
    
    chodf = chodf.rename(columns={'a':'年齢', 'b':'人口'})
    chodf['年齢'] = chodf['年齢'].astype(int)
    
    meg_tmp = pd.concat([meg_tmp, chodf])

meg_finedata = meg_tmp[['町丁目名','年齢','人口']]
meg_finedata = pd.merge(meg_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

meg_finedata['区']='目黒区'

for i in d2k.keys():
    meg_finedata['町丁目名']=meg_finedata['町丁目名'].str.replace(i,d2k[i])


In [16]:
meg_finedata.head()

,町丁目名,年齢,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区
0,駒場一丁目,0,30,13110,目黒区,131100027001,駒場１丁目,目黒区
1,駒場一丁目,1,16,13110,目黒区,131100027001,駒場１丁目,目黒区
2,駒場一丁目,2,26,13110,目黒区,131100027001,駒場１丁目,目黒区
3,駒場一丁目,3,19,13110,目黒区,131100027001,駒場１丁目,目黒区
4,駒場一丁目,4,32,13110,目黒区,131100027001,駒場１丁目,目黒区


## 4. 港区

In [17]:
import tabula

mina_file = prefix + '港区2022年4月.pdf'

# データはTabulaで抽出、町丁目名は次のセルで抽出。
mina_data = tabula.read_pdf(mina_file,pages="1-117",stream=True)


In [18]:
textfile = open(prefix+'港区町丁目名.txt','r')
output_text = textfile.read()
textfile.close()

import re

pattern = '(.+丁目)|(麻布.+町)|(麻布狸穴町)$'

minacho = re.findall(pattern, output_text)

from itertools import chain
def flatten(nested_list):
    return list(chain.from_iterable(nested_list))

#flatten(list(map(lambda x: list(x), minacho)))

chomeilist = list(map((lambda y: ''.join(y)), list(map(lambda x: list(x), minacho))))

In [19]:
mina_finedata = pd.DataFrame()

for i in range(0,116):
    if i != 39:
        k = mina_data[i].iloc[:21,0:3].set_axis(['世代','a','b'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k = k.fillna(0)
        k = k.astype({'a':str, 'b':str})
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)
        k['b'] = k['b'].str.replace(',','')
        k['b'] = k['b'].str.replace(r'0\.0','0', regex=True)
    else:
        k = mina_data[i].iloc[:21,0:2].set_axis(['世代','a'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)        
        k['b'] = '0'
        
    mina_finedata = pd.concat([mina_finedata, k])

mina_finedata = mina_finedata.astype({'a':int, 'b':int}).reset_index(drop=True) 

mina_finedata['人口'] = mina_finedata['a'] + mina_finedata['b']
#

for r in mina_finedata.itertuples():
    if not(((r.a == 0) and (r.b==r.人口)) or ((r.b==0) and (r.a==r.人口))):
        print(r.町丁目名, ' ', r.a, r.b)

mina_finedata = mina_finedata.drop(columns=['a','b'])[['町丁目名','世代','人口']]
mina_finedata = pd.merge(mina_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

mina_finedata['区']='港区'

for i in d2k.keys():
    mina_finedata['町丁目名']=mina_finedata['町丁目名'].str.replace(i,d2k[i])



In [20]:
mina_finedata.head()

,町丁目名,世代,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区
0,赤坂一丁目,0~4歳,35,13103,港区,131030008001,赤坂１丁目,港区
1,赤坂一丁目,5~9歳,37,13103,港区,131030008001,赤坂１丁目,港区
2,赤坂一丁目,10~14歳,25,13103,港区,131030008001,赤坂１丁目,港区
3,赤坂一丁目,15~19歳,22,13103,港区,131030008001,赤坂１丁目,港区
4,赤坂一丁目,20~24歳,20,13103,港区,131030008001,赤坂１丁目,港区


In [21]:
#mina_finedata.to_excel(prefix + dataname + '（港）.xlsx', index=False)

## 5. 渋谷

In [22]:
shib_data = pd.read_excel(prefix + '渋谷区2022年1月_rev.xlsx')

shib_finedata = shib_data[['町丁名','年齢','人口']].rename(columns={'町丁名':'町丁目名'})

shib_finedata = pd.merge(shib_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

shib_finedata['区']='渋谷区'

for i in d2k.keys():
    shib_finedata['町丁目名']=shib_finedata['町丁目名'].str.replace(i,d2k[i])


In [23]:
shib_finedata.groupby(['町丁目名',
                       '市区町村コード',
                       '市区町村名',
                       '大字町丁目コード',
                       '大字町丁目名',
                       '区'])['人口'].sum().reset_index().head()

,町丁目名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区,人口
0,上原一丁目,13113,渋谷区,131130008001,上原１丁目,渋谷区,2961
1,上原三丁目,13113,渋谷区,131130008003,上原３丁目,渋谷区,3203
2,上原二丁目,13113,渋谷区,131130008002,上原２丁目,渋谷区,4168
3,代々木一丁目,13113,渋谷区,131130002001,代々木１丁目,渋谷区,2856
4,代々木三丁目,13113,渋谷区,131130002003,代々木３丁目,渋谷区,4651


In [24]:
shib_finedata.head()

,町丁目名,年齢,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区
0,恵比寿一丁目,0,25,13113,渋谷区,131130010001,恵比寿１丁目,渋谷区
1,恵比寿一丁目,1,36,13113,渋谷区,131130010001,恵比寿１丁目,渋谷区
2,恵比寿一丁目,2,31,13113,渋谷区,131130010001,恵比寿１丁目,渋谷区
3,恵比寿一丁目,3,37,13113,渋谷区,131130010001,恵比寿１丁目,渋谷区
4,恵比寿一丁目,4,17,13113,渋谷区,131130010001,恵比寿１丁目,渋谷区


In [25]:
#shib_finedata.to_excel(prefix + dataname + '（渋谷）.xlsx', index=False)

## 6. 台東

In [26]:
##
## 上のセルで、OCRからデータを抽出し、手作業で値の確認を行った。
##　町丁目名年齢年代別人口分布（台東）その'+i+'_確認済み.xlsx　がその結果。
## それらを統合してデータを作る。
##
tai_data = pd.DataFrame()
for i in ['１','２','３']:
    tai = pd.read_excel(prefix+'町丁目名年齢年代別人口分布（台東）その'+i+'_確認済み.xlsx')
    tai_data = pd.concat([tai_data, tai])

tai_data['区']='台東区'
tai_data = tai_data.drop(columns='行政コード')

tc = {'level_2':'世代', 0:'人口'}
tmp_tai = tai_data.set_index(['区','町丁目名'])
tt = pd.DataFrame(tmp_tai.stack().reset_index()).rename(columns=tc)
tai_data = pd.merge(tt, c_code, left_on=['区','町丁目名'], right_on=['市区町村名','大字町丁目名'])

#for i in d2k.keys():
#    tai_data['町丁目名']=tai_data['町丁目名'].str.replace(i,d2k[i])
    
tai_finedata = tai_data

In [27]:
#tai_finedata.to_excel(prefix + dataname + '（台東）.xlsx', index=False)

## 7. 中央

In [28]:
chu_data = pd.read_excel(prefix + '中央区2022年3月.xlsx',
                         dtype={'丁目':str}).rename(columns={'総数':'人口'})

chu_data = chu_data.fillna('')
chu_data['町丁目名']=''

for i, r in chu_data.iterrows():
    k = r['丁目']
    if k!='':
        chu_data.loc[i,'町丁目名'] = r['地域'] + sd2k[str(k) + '丁'] +'目'
    else:
        chu_data.loc[i,'町丁目名'] = r['地域']

chu_data['世代']='全世代'
chu_data['区'] = '中央区'
chu_data['gord'] = 99

#chu_data = pd.merge(chu_data, c_code, left_on='町丁目名', right_on='大字町丁目名')
chu_finedata = chu_data[['区','町丁目名','世代','人口']]

In [29]:
chu_finedata.head()

,区,町丁目名,世代,人口
0,中央区,八重洲２丁目,全世代,35
1,中央区,京橋１丁目,全世代,66
2,中央区,京橋２丁目,全世代,106
3,中央区,京橋３丁目,全世代,68
4,中央区,銀座１丁目,全世代,1473


In [30]:

'銀座１丁目'.replace('３丁目','三丁目')

'銀座１丁目'

In [31]:
for k in d2k.keys():
    for i,r in chu_finedata.iterrows():
        chu_finedata.loc[i,'町丁目名']=(r.町丁目名).replace(k, d2k[k])

In [36]:
chu_finedata

,区,町丁目名,世代,人口
0,中央区,八重洲二丁目,全世代,35
1,中央区,京橋一丁目,全世代,66
2,中央区,京橋二丁目,全世代,106
3,中央区,京橋三丁目,全世代,68
4,中央区,銀座一丁目,全世代,1473
...,...,...,...,...
93,中央区,晴海一丁目,全世代,4128
94,中央区,晴海二丁目,全世代,7164
95,中央区,晴海三丁目,全世代,5080
96,中央区,晴海四丁目,全世代,110


In [33]:
#chu_finedata.to_excel(prefix + dataname + '（中央）.xlsx', index=False)

In [34]:
#chu_finedata.to_csv(prefix + 'final_chu_data.csv', index=False)
#chu_finedata.to_csv(prefix + dataname + '（渋谷）.xlsx', index=False)

## データ統合

In [35]:
finedata = pd.concat([ohta_finedata,
                      meg_finedata,
                      shib_finedata,
                      shina_finedata,
                      ch_fd,
                      ner_fd
                      ]).reset_index(drop=True)

finedata['世代'] = ''

#finedata = pd.concat([finedata, chu_finedat#a])

# fiveband = list(mina_finedata.loc[:20,:]['世代'])
#print(fiveband)

# ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
# '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
# '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳'
# '105~109歳', '110~114歳', '115~119歳', '120歳以上']

# 0~130まで5づつでビンを作成
bin_band =  list(range(0,131,5))
# binのラベル
fiveband = ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
 '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
 '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳',
 '105~109歳', '110~114歳', '115~119歳', '120~124歳', '125~129歳']#, '130歳以上']

fiveband_df = pd.DataFrame(
            {
                'gord' : list(range(0,26)),
                '世代': fiveband
            }
)


finedatas = finedata[finedata['人口']>0]

finedata['世代']=pd.cut(finedata['年齢'], bin_band, right=False, labels=fiveband)

finedata = pd.merge(finedata, fiveband_df, on='世代', how='left')

#finedata = finedata[~(finedata['人口']==0)]

In [37]:
finedata

,町丁目名,年齢,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区,世代,gord
0,大森東一丁目,0,24,13111,大田区,131110039001,大森東１丁目,大田区,0~4歳,0
1,大森東一丁目,1,28,13111,大田区,131110039001,大森東１丁目,大田区,0~4歳,0
2,大森東一丁目,2,36,13111,大田区,131110039001,大森東１丁目,大田区,0~4歳,0
3,大森東一丁目,3,36,13111,大田区,131110039001,大森東１丁目,大田区,0~4歳,0
4,大森東一丁目,4,31,13111,大田区,131110039001,大森東１丁目,大田区,0~4歳,0
...,...,...,...,...,...,...,...,...,...,...
102286,西大泉町,106,0,13120,練馬区,131200046000,西大泉町,練馬区,105~109歳,21
102287,西大泉町,107,0,13120,練馬区,131200046000,西大泉町,練馬区,105~109歳,21
102288,西大泉町,108,0,13120,練馬区,131200046000,西大泉町,練馬区,105~109歳,21
102289,西大泉町,109,0,13120,練馬区,131200046000,西大泉町,練馬区,105~109歳,21


In [38]:
#fd_grp = ['区','町丁目名','大字町丁目コード','世代','gord']
fd_grp = ['区','町丁目名','大字町丁目コード','世代','gord']
srt = ['大字町丁目コード','gord']

final = pd.DataFrame(finedata.groupby(fd_grp)['人口'].agg(sum)).sort_values(srt).reset_index()

mina_final = pd.merge(mina_finedata, fiveband_df, on='世代')[fd_grp+['人口']]
tai_final = pd.merge(tai_finedata, fiveband_df, on='世代')[fd_grp+['人口']]
st_final = pd.merge(st_fd, fiveband_df, on='世代')[fd_grp+['人口']]

final = pd.concat([final, mina_final, tai_final, st_final])

In [39]:
final

,区,町丁目名,大字町丁目コード,世代,gord,人口
0,千代田区,内幸町１丁目,131010001001,0~4歳,0,0
1,千代田区,内幸町１丁目,131010001001,5~9歳,1,0
2,千代田区,内幸町１丁目,131010001001,10~14歳,2,0
3,千代田区,内幸町１丁目,131010001001,15~19歳,3,0
4,千代田区,内幸町１丁目,131010001001,20~24歳,4,0
...,...,...,...,...,...,...
3460,世田谷区,南烏山５丁目,131120014005,100~104歳,20,2
3461,世田谷区,南烏山６丁目,131120014006,100~104歳,20,1
3462,世田谷区,北烏山１丁目,131120016001,100~104歳,20,2
3463,世田谷区,北烏山２丁目,131120016002,100~104歳,20,2


In [40]:
#　エラーが出る町丁目名
final['町丁目名'] = final['町丁目名'].str.replace('松濤','松涛')
final['町丁目名'] = final['町丁目名'].str.replace('鶯谷町','鴬谷町')

In [41]:
final = final.astype({'大字町丁目コード':str})

In [42]:
d2k = {
    '１丁':'一丁',
    '２丁':'二丁',
    '３丁':'三丁',
    '４丁':'四丁',
    '５丁':'五丁',
    '６丁':'六丁',
    '７丁':'七丁',
    '８丁':'八丁',
    '９丁':'九丁'
}

for i in d2k.keys():
    final['町丁目名']=final['町丁目名'].str.replace(i,d2k[i])


In [43]:
final

,区,町丁目名,大字町丁目コード,世代,gord,人口
0,千代田区,内幸町一丁目,131010001001,0~4歳,0,0
1,千代田区,内幸町一丁目,131010001001,5~9歳,1,0
2,千代田区,内幸町一丁目,131010001001,10~14歳,2,0
3,千代田区,内幸町一丁目,131010001001,15~19歳,3,0
4,千代田区,内幸町一丁目,131010001001,20~24歳,4,0
...,...,...,...,...,...,...
3460,世田谷区,南烏山五丁目,131120014005,100~104歳,20,2
3461,世田谷区,南烏山六丁目,131120014006,100~104歳,20,1
3462,世田谷区,北烏山一丁目,131120016001,100~104歳,20,2
3463,世田谷区,北烏山二丁目,131120016002,100~104歳,20,2


In [44]:
final.to_csv(prefix+'final_data.csv', encoding='utf-8', index=False)
#chu_finedata.to_csv(prefix+'final_chu_data.csv', encoding='utf-8', index=False)
final.to_excel(prefix+"final.xlsx",index=False)